<a href="https://colab.research.google.com/github/shinjihirako13-vis/Elevate-Lab-Project/blob/main/Elevate_Lab_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from google.colab import files
import IPython.display as ipd

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
def extract_features(file_path):
    audio, sr = librosa.load(file_path, duration=30)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

In [ ]:
eatures = []
labels = []

for genre in genres:
    genre_path = os.path.join(data_path, genre)
    for file in os.listdir(genre_path):
        if file.endswith(".wav"):
            try:
                mfcc = extract_features(os.path.join(genre_path, file))
                features.append(mfcc)
                labels.append(genre)
            except Exception as e:
                print("Error processing", file, ":", e)


In [ ]:
X = np.array(features)
le = LabelEncoder()
y = to_categorical(le.fit_transform(labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(40,), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
epochs = 30
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
model.save("genre_classification_model.h5")


In [ ]:
def predict_genre(file):
    audio, sr = librosa.load(file, duration=30)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfcc_mean = np.mean(mfcc.T, axis=0)
    mfcc_mean = mfcc_mean.reshape(1, -1)
    prediction = model.predict(mfcc_mean)[0]
    predicted_index = np.argmax(prediction)
    confidence = prediction[predicted_index]
    return le.inverse_transform([predicted_index])[0], confidence


In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
    genre, confidence = predict_genre(fn)
    print(f"Predicted Genre: {genre}, Confidence: {confidence:.2f}")
    ipd.display(ipd.Audio(fn))

In [ ]:
def show_spectrogram(file):
    audio, sr = librosa.load(file, duration=30)
    X = librosa.stft(audio)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(14, 5))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()
    plt.title("Spectrogram")
    plt.show()